In [1]:
#读取指定点的S1数据，基于gee和geemap
import ee
import geemap
import os
from osgeo import gdal
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
Map = geemap.Map()


Successfully saved authorization token.


In [5]:
#读取SOC数据并生成ee.feactureColection
import pandas as pd
wb= pd.read_excel('./final_data.xlsx')
print(wb.keys())
features=[]
for i in range(wb.shape[0]):
    if pd.notnull(wb['0-10cm content %'][i]):
        lat = wb['Lat'][i]
        long = wb['Long'][i]
        points = ee.Geometry.Point([long,lat])
        features.append(ee.Feature(points,{'0-10cm content':wb['0-10cm content %'][i]}))

fc=ee.FeatureCollection(features)
Map.addLayer(fc,{'color':'FF0000'},'Points')

Index(['General', 'Lat', 'Long', '0-10cm density kgC m-2 cm-1',
       '0-10cm content %', '10-30cm density kgC m-2 cm-1',
       '10-30cm content %'],
      dtype='object')


In [3]:
#roi覆盖阿拉斯加区域
roi = ee.Geometry.Rectangle([[-180.351563,50.513427],[ -127.617188,71.965388]],geodesic=False,proj='EPSG:4326')
Map.addLayer(roi,{'color':'ff0000','fillColor':'000000','width':3},'ROI')
date = ee.Filter.date('2014-01-01','2016-01-01')
image_collection=ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.bounds(roi)).filter(date)

imgVV=image_collection.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW')).select('VV').mean()
imgVH = image_collection.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).select('VH').mean()
incidence_angle=image_collection.select("angle").max()

Map.addLayer(imgVV,{'min': -25, 'max': 1},'S1-VV')
Map.addLayer(imgVH,{'min': -25, 'max': 1},'S1-VH')
# Map.addLayer(incidence_angle,{'min':0,'max':90},'incidence angle')
#现在还没想到incidence angle要怎么处理

#融合VV VH波段
img=imgVV.addBands(imgVH)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
#采样
import numpy as np
fc2=img.sampleRegions(collection=fc,scale=50,geometries=True)
fc2=fc2.toList(fc2.size().getInfo())
fc2=fc2.getInfo()
fc2=np.array(fc2)
header=['Lat','Long','VH','VV','C-density']
df=pd.DataFrame(columns=header)
for i in range (fc2.shape[0]):
    df_temp=pd.DataFrame([[fc2[i]['geometry']['coordinates'][1],fc2[i]['geometry']['coordinates'][0],fc2[i]['properties']['VH'],fc2[i]['properties']['VV'],fc2[i]['properties']['C-density']]],columns=header)
    df=pd.concat([df,df_temp],ignore_index=True,axis=0)

In [5]:
df.to_excel('data_toplayer_with_VVVH.xlsx',index=False)

In [ ]:
#改写geemap的fishnet函数，使之能适配
from geemap.common import latlon_grid,ee_export_vector
def fishnet(
    data,
    h_interval=1.0,
    v_interval=1.0,
    rows=None,
    cols=None,
    delta=1.0,
    intersect=True,
    output=None,
    **kwargs,
):
    
    
    coords = data.getInfo()[0]['coordinates']

    west = coords[0][0][0]
    east = coords[0][1][0]
    south = coords[0][0][1]
    north = coords[0][2][1]
    print(west)


    if rows is not None and cols is not None:
        v_interval = (north - south) / rows
        h_interval = (east - west) / cols

    # west = west - delta * h_interval
    east = east + delta * h_interval
    # south = south - delta * v_interval
    north = north + delta * v_interval

    grids = latlon_grid(v_interval, h_interval, west, east, south, north)

    if intersect:
        grids = grids.filterBounds(roi)

    if output is not None:
        ee_export_vector(grids, output)

    else:
        return grids



fishnet = fishnet(roi.geometries(), rows=16, cols=16, delta=0.5)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fishnet.style(**style), {}, 'Fishnet')


geemap.download_ee_image_tiles(
    imgVV, fishnet, './download', prefix="S1VV", crs="EPSG:4326", scale=500
)


In [1]:
#加载数据集
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
df=pd.read_excel('./data_toplayer_with_VVVH.xlsx')
inx=[]
for i in range(df.shape[0]):
    if df['C-density'][i]!=0:
        inx.append(i)
df=df.iloc[inx,:]
X = pd.concat([df['VH'],df['VV']],axis=1)
y = df['C-density']
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,shuffle=True)

In [2]:
#模型训练
model = RandomForestRegressor()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
MSE=mean_squared_error(y_test,y_pred)
R2=model.score(X_test,y_test)
print("MSE:\t",MSE)
print('R2:\t',R2)

MSE:	 0.5121546034139727
R2:	 0.3541209274202418


In [3]:
#模型使用

from osgeo import gdal

filePath = 'D:/sentinel-1/test/50m_res.tif'  # tif文件路径
dataset = gdal.Open(filePath)  # 打开tif

adfGeoTransform = dataset.GetGeoTransform()  # 读取地理信息
# geo_information(0):左上像素左上角的x坐标。
# geo_information(1):w - e像素分辨率 / 像素宽度。
# geo_information(2):行旋转（通常为零）。
# geo_information(3):左上像素左上角的y坐标。
# geo_information(4):列旋转（通常为零）。
# geo_information(5):n - s像素分辨率 / 像素高度（北半球上图像为负值）

nXSize = dataset.RasterXSize  # 列数
nYSize = dataset.RasterYSize  # 行数
bands=dataset.RasterCount
print(adfGeoTransform)
print('colum:',nXSize)
print('row',nYSize)

(-157.44683514188839, 0.00044915764205976077, 0.0, 64.26571901103478, 0.0, -0.00044915764205976077)
colum: 12716
row 4285


In [4]:
rawVH=dataset.GetRasterBand(1).ReadAsArray()
rawVV=dataset.GetRasterBand(2).ReadAsArray()


In [7]:
print(rawVH.shape)

(4285, 12716)


In [6]:
import numpy as np
import pandas as pd
print(np.max(rawVH))
print(np.min(rawVH))

VH=pd.DataFrame(rawVH.reshape(nXSize*nYSize,1))
VV=pd.DataFrame(rawVV.reshape(nXSize*nYSize,1))

X=pd.concat([VH,VV],ignore_index=True,axis=1)
y=model.predict(X)

8.971328
-55.134724


c:\Users\98143\.conda\envs\gee\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [9]:
y_raster=y.reshape(nYSize,nXSize)

driver = gdal.GetDriverByName("GTiff")
New_YG_dataset = driver.Create('./SOC.tif', nXSize, nYSize, 1, gdal.GDT_Float32)
New_YG_dataset.SetGeoTransform(dataset.GetGeoTransform())
New_YG_dataset.SetProjection(dataset.GetProjection())
band = New_YG_dataset.GetRasterBand(1)
print(New_YG_dataset)
band.WriteArray(y_raster)
# 这里注意，创建完成之后，需要删除，不然，后面再次打开的时候，会被占用
del New_YG_dataset

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001CDA6F81200> >
